In [ ]:
from IPython import get_ipython
my_variables = set(dir())  # Write this line at the beginning of cell
my_variables = list(set(dir()) - my_variables)  # Write these 2 lines at the end of cell
get_ipython().magic('%reset_selective -f [{}]'.format(','.join(my_variables)))

import cv2 as cv
import numpy as np
import numpy.ma as ma
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw 
import pylab
# print(bublick_big)

In [ ]:
def plt_show(image, color = None):
    pic_size=5
    plt.figure(figsize = (pic_size, pic_size))
    plt.imshow(image, color)
    plt.show()
   

In [ ]:
def coinsNumberFind(makers):
    l = makers.reshape(len(makers)*len(makers[0])).tolist()
    print(type(l), (set(l)))
    return (len(set(l))-1)

In [ ]:
def otsuFragment(pix, old):
    hist = hist_ar(pix)

    summ = 0
    for x in range(len(pix)):
        summ = summ +pix[x]

    indexMax = 0
    sigMax = 0.0 

    l1 = 0 
    o1 = 0
    
    if len(old)==0:
        old_new=[0]

    for thresh in range(1,len(hist)): 
       
        if not thresh in old:
            l1 = l1 +  hist[thresh]
            
            # print("o1 ",o1)
            o1 =o1+ thresh * hist[thresh]
    
            # print("l1",l1)
            omega1 = l1 /len(pix) 
            omega2 = 1.0 - omega1 
    
            lyamda1 = o1 /l1
            lyamda2 = (summ - o1)/(len(pix) - l1)
            
            between = lyamda1 - lyamda2
    
            sigma = omega1 * omega2 * between * between

            if (sigma > sigMax): 
                sigMax = sigma
                indexMax = thresh
    return indexMax 

def hist_ar(pixarr):
        h = np.zeros(256, np.float32)
        for x in range(len(pixarr)):
            h[round(pixarr[x])]=h[round(pixarr[x])]+1
        for i in range(len(h)):
            if len(h)>i:
                if ((h[i]==0) & (len(h)>0)):
                    h=np.setdiff1d(h, 0)
                    i = i - 1
        return h
    
def otsu_loc(image_name):
    partsH = 2;  partsV = 2
    
    ref = cv.imread(image_name, cv.IMREAD_GRAYSCALE)
    image = Image.open(image_name)
    new_pix = ImageDraw.Draw(image)  
    
    sizeV = image.size[1];  sizeH = image.size[0]

    stepV = np.array([x for x in range(0,sizeV-sizeV//partsH+1,int(sizeV//partsH))])  
    stepV=np.append(stepV,sizeV)
    
    stepH = np.array([x for x in range(0,sizeH-sizeH//partsV+1,int(sizeH//partsV))])    
    stepH=np.append(stepH, sizeH)

    pixarr = cv.imread(image_name, cv.IMREAD_GRAYSCALE)
    for i in range(len(stepV)-1):
        for j in range(len(stepH)-1):
            arr1 = arr = (pixarr[stepV[i]:stepV[i+1],stepH[j]:stepH[j+1]])
            limit = otsuFragment(arr1.reshape(len(arr)*len(arr[0])), [])
            for x in range(stepV[i],stepV[i+1]):
                for y in range(stepH[j],stepH[j+1]):
                    if arr[x-stepV[i],y-stepH[j]] < limit:
                        pix = 0
                    else:
                        pix = 255
                    ref[x][y]=pix
                    new_pix.point((y, x), (pix, pix, pix)) 
    fig, ax = plt.subplots(1,2)
    plt.title("Local binarization Otsu's method")
    ax[0].imshow( Image.open(image_name) )
    ax[1].imshow(image)
    plt.show()
    return ref

In [ ]:
picture = cv.imread("pic3.jpg", cv.IMREAD_GRAYSCALE)
plt_show(picture, "gray")

In [ ]:
_, picture = cv.threshold(picture,130,255,cv.THRESH_BINARY_INV)

plt_show(picture, "gray")


In [ ]:
bin_ref = otsu_loc("pic3.jpg")

In [ ]:
circle = cv.getStructuringElement(cv.MORPH_ELLIPSE, (9, 2))# np.ones((9,2),np.uint8)
all_coins = cv.dilate(bin_ref, circle)

plt_show(all_coins)

# for i in range(1,10):
#     for j in range(1,10):
#         circle = np.ones((i,j),np.uint8)
#         small_coins = cv.dilate(bin_ref, circle)
#         print(i,j)
#         plt_show(small_coins);plt.show()

In [ ]:
circle = np.ones((25,25),np.uint8)
small_coins = cv.erode(all_coins, circle)
plt_show(small_coins)

circle = np.ones((10,14),np.uint8)
small_coins = cv.erode(small_coins, circle)
plt_show(small_coins)



circle = cv.getStructuringElement(cv.MORPH_ELLIPSE, (40, 37))
small_coins = cv.dilate(small_coins, circle)
plt_show(small_coins)


# for i in range(25,37):
#     for j in range(25,37):
#         circle = np.ones((i,j),np.uint8)
#         small_coins = cv.erode(all_coins, circle)
# #         print(i,j)
# #         plt_show(small_coins);plt.show()
#         for c in range(2,15):
#             for d in range(2,15):
#                 circle = np.ones((c,d),np.uint8)
#                 small_coins_double_dilate = cv.erode(small_coins, circle)
#                 print(i,j, c,d)
#                 plt_show(small_coins_double_dilate);plt.show()

In [ ]:
_, markers_for_small_coins = cv.connectedComponents(small_coins)

plt_show(markers_for_small_coins)

In [ ]:
print("countSmallCoins  =  ", coinsNumberFind(markers_for_small_coins))

In [ ]:
circle = cv.getStructuringElement(cv.MORPH_ELLIPSE, (30, 20))# np.ones((20,24),np.uint8)
small_coinsDilate = cv.dilate(small_coins, circle)
plt_show(small_coinsDilate)


In [ ]:
bublick = abs(small_coinsDilate-small_coins)
plt_show(bublick)

In [ ]:
_, markers_for_bublick = cv.connectedComponents(small_coinsDilate)


markers_for_bublick = markers_for_bublick + 1
markers_for_bublick[bublick==255] = 0


plt_show(markers_for_bublick)


reference = cv.imread("pic3.jpg")
markers_for_bublick = cv.watershed(reference, markers_for_bublick)


plt_show(markers_for_bublick)

reference[markers_for_bublick == -1] = [255, 0, 0]


plt_show(reference)

In [ ]:
circle = np.ones((130,130),np.uint8)
small_coins_double_dilate = cv.dilate(small_coins, circle)

plt_show(small_coins_double_dilate)

In [ ]:
big_coins=all_coins.copy()
for i in range(len(small_coins_double_dilate)):
    for j in range(len(small_coins_double_dilate[0])):
        if (small_coins_double_dilate[i][j]!=0):
            big_coins[i][j]=0
plt_show(big_coins)

circle = cv.getStructuringElement(cv.MORPH_ELLIPSE, (15, 10))
big_coinser = cv.dilate(big_coins, circle)

In [ ]:
_, markers_for_big_coins = cv.connectedComponents(big_coinser)
plt_show(markers_for_big_coins)

In [ ]:
print("countBigCoins  =  ", coinsNumberFind(markers_for_big_coins))

In [ ]:
circle = cv.getStructuringElement(cv.MORPH_ELLIPSE, (45, 40))# np.ones((20,24),np.uint8)
big_coins_dil = cv.dilate(big_coins, circle)
plt_show(big_coins_dil)

circle = cv.getStructuringElement(cv.MORPH_ELLIPSE, (55, 55))# np.ones((20,24),np.uint8)
big_coins_erode = cv.erode(big_coins_dil, circle)
plt_show(big_coins_erode)


bublick_big = abs(big_coins_dil - big_coins_erode)
plt_show(bublick_big)

In [ ]:
circle = cv.getStructuringElement(cv.MORPH_ELLIPSE, (5, 5))#
plt_show(big_coins)

big_coins5 = cv.dilate(big_coins, circle)
plt_show(big_coins5)

_, markers_for_bublick2 = cv.connectedComponents(big_coins5)

markers_for_bublick2 = markers_for_bublick2 + 1
markers_for_bublick2[bublick_big==255] = 0
plt_show(markers_for_bublick2)


reference = cv.imread("pic3.jpg")
markers_for_bublick2 = cv.watershed(reference, markers_for_bublick2)
plt_show(markers_for_bublick2)

reference[markers_for_bublick2 == -1] = [0, 191, 255]
plt_show(reference)

In [ ]:
circle = np.ones((5,5),np.uint8)
coins_erode = cv.erode(all_coins, circle)

for i in range(len(coins_erode)):
    for j in range(len(coins_erode[0])):
        if (big_coins_dil[i][j]!=0):
            coins_erode[i][j]=big_coins_dil[i][j]
    
plt_show(coins_erode)


_, markers_for_bublick_all = cv.connectedComponents(coins_erode)
print("AllCoins  =  ", coinsNumberFind(cv.connectedComponents(coins_erode)[1]))

plt_show(all_coins)

bublick_all = bublick.copy()
for i in range(len(bublick_all)):
    for j in range(len(bublick_all[0])):
        if (bublick_big[i][j]!=0):
            bublick_all[i][j]=bublick_big[i][j]            
plt_show(bublick_all)


markers_for_bublick_all = markers_for_bublick_all + 1
markers_for_bublick_all[bublick_all==255] = 0
plt_show(markers_for_bublick_all)


reference = cv.imread("pic3.jpg")
markers_for_bublick_all = cv.watershed(reference, markers_for_bublick_all)
plt_show(markers_for_bublick_all)


reference[markers_for_bublick_all == -1] = [252, 15, 192]
plt_show(reference)